# Import Dependencies

In [1]:
import numpy as np
import random
import operator
# suppress scientific notation printing in numpy 
np.set_printoptions(suppress=True)


# Initializations

In [2]:
actions = np.array(["^", ">", "v", "<"])

In [3]:
# a dictionary that stores the rewards for all the actions in every possible state
moves = {
  "0": { "^": ["0", -10], "v": ["3", -1], "<": ["0", -10], ">": ["1", -1] },
  "1": { "^": ["1", -10], "v": ["4", -1], "<": ["0", -1],  ">": ["2", -1] },
  "2": {"^":  ["2", -10], "v": ["5", -1], "<": ["1", -1],  ">": ["2", -10]},
  "3": {"^":  ["0", -1],  "v": ["6", -1], "<": ["3", -10], ">": ["4", -1] },
  "4": {"^":  ["1", -1],  "v": ["7", -1], "<": ["3", -1],  ">": ["5", -1] },
  "5": {"^":  ["2", -1],  "v": ["8", -1], "<": ["4", -1],  ">": ["5", -10]},
  "6": {"^":  ["3", -1],  "v": ["6", -10],"<": ["6", -10], ">": ["7", -1] },
  "7": {"^":  ["4", -1],  "v": ["7", -10],"<": ["6", -1],  ">": ["8", -1] },
  "8": {"^":  ["5", -1],  "v": ["8", -10],"<": ["7", -1],  ">": ["8", -10]}}

In [4]:
# each row here is a state and the column correspond to actions as they are ordered in the above actions array
policy = np.ones((9,4))*0.25
policy

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

In [5]:
# each row here is a state and the column correspond to actions as they are ordered in the above actions array
Q = np.ones((9,4))*-9999
Q

array([[-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.],
       [-9999., -9999., -9999., -9999.]])

In [6]:
start_states = np.arange(8).astype(str)
start_states

array(['0', '1', '2', '3', '4', '5', '6', '7'], dtype='<U21')

In [7]:
final_state = np.array([8]).astype(str)
final_state

array(['8'], dtype='<U21')

In [8]:
start_prob = np.array([0.125 for i in start_states])
start_prob

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [9]:
# each row here is a state and the column correspond to actions as they are ordered in the above actions array
# each entry is a tuple where the first element of the tuple is the sums so far and the second element of
# the tuple is the number of sums far
rewards = [[(0,0)for action in range(4)] for state in range(9)]
rewards

[[(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0)]]

# Functions

In [10]:
def action_offset(action):
    """takes your requested action and adds an offset.
       80% probability for 0 offset, 10% chance for left offset, 10% chance for right offset
       """    
    offset = np.random.choice([-1,0,1], 1, p=[0.1,0.8,0.1])
    return actions[ (int(np.where(actions == action)[0]) + offset) % len(actions)][0]   
 

In [11]:
def transition(state, policy):
    """ simulates the transition from the input state using the input poicy and 
        returns reward, new_state, requested_action, actual_action
    """
    requested_action = np.random.choice( actions, 1, p=policy[int(state)])
    actual_action = action_offset(requested_action)
    new_state, reward = moves[state][actual_action]
    return reward, new_state, requested_action[0], actual_action

In [12]:
def one_episode(state, policy, actions, print_flag=True):
    """ simuates a single episode starting from the input state and using the input policy 
        until it reaches the final state. It returns the trace for the episode and the rewards 
        at each line of the episode
    """
    trace = []
    rewards_list = []
    # first randomly chosen state action pair
    requested_action = random.choice(actions)
    actual_action = action_offset(requested_action)
    new_state, reward = moves[state][actual_action]
    if print_flag:
        print("Starting in State ", state)
        state = new_state
        print((reward, new_state, requested_action, actual_action))
    while state != final_state[0]:
        reward, new_state, requested_action, actual_action = transition(state, policy)
        if print_flag:
            print((reward, new_state, requested_action, actual_action))
        trace.append((reward, state, requested_action, actual_action))
        rewards_list.append(reward)
        state = new_state
    trace.append((reward, state, requested_action, actual_action))
    if print_flag:
        print(" FINAL STATE REACHED!!!")
    return trace, rewards_list

In [13]:
def G(trace, rewards_list, line):
    """ sum all the rewards after the first occurence of input line in the input list
        returns the number of rewards summed and total sum of rewards
    """
    first_occur = trace.index(line)+1
    after_first_occur = rewards_list[first_occur:]
    return sum(after_first_occur)
#     return (sum(after_first_occur), len(after_first_occur))

In [14]:
def accumulate_rewards(rewards, trace, rewards_list):
    """ populates the previously empty rewards matrix with the rewards of the state-action pairs in the input trace 
    """
    # this first loop gets the rewards for each state action pair into a dictionary to avoid duplicates
    state_action_pair_reward = {}
    for line in trace:      
        state_action_pair = (int(line[1]), int(np.where(actions==line[2])[0]))                            
        if state_action_pair not in state_action_pair_reward:
            state_action_pair_reward[state_action_pair] = G(trace, rewards_list, line)

    # this second loop populates the rewards matrix using the dictionary
    for state_action_pair in state_action_pair_reward:
        state, action = state_action_pair
        rewards[state][action] = tuple(map(operator.add,rewards[state][action],
                                           (state_action_pair_reward[state_action_pair],1)))
#         rewards[state][action] = tuple(map(operator.add,rewards[state][action],
#                                            state_action_pair_reward[state_action_pair]))
    return rewards

In [15]:
def updateQ(Q, rewards):
    """ updates and returns Q based on the input trace
    """    
    # this loop replaces each entry of Q with the average of the corresponding non-empty list entry in 
    # rewards matrix
    for state in range(len(rewards)):
        for action in range(len((actions))):
            if rewards[state][action][1] !=0:
                Q[state][action] = rewards[state][action][0]/rewards[state][action][1]
    return Q    

In [16]:
def updatePolicy(policy, Q):
    """ updates and returns the input policy matrix based on the input Q matrix
    """
    for state in range(len(policy)):
        optimal_action = np.argmax(Q[state])
        # create a boolean mask to replace the non-optimal action probailities with 0 and the optimal probaility with 1
        mask = np.zeros((1,4),dtype=bool)[0]
        mask[optimal_action] = True
        # assign optimal action a probablity of 1 and all other actions a probability of 0
        policy[state][mask] = 1 
        policy[state][~mask] = 0
    return policy

In [17]:
def visualize(policy):
    """ takes in a 9x4 matrix of 0's and 1's where rows correspond to states and columns correspond to actions
        returns a 3x3 matrix containing the policy's suggested actions at each cell
    """
    visualized = np.zeros((3,3)).astype(str)
    rows = visualized.shape[0]
    cols = visualized.shape[1]
    for row in range(rows):
        for col in range(cols):
            visualized[row][col] = actions[np.argmax(policy[row*rows+col])]
    visualized[row][col] = 'T'
    return visualized

# Monte Carlo Method with Exploring Starts

## Part I

In [18]:
def MCMES1(policy, Q, rewards, num_episodes):
    """ 
        inputs: policy, Q and rewards are the initialized policy, Q and rewards matrix
        This functions uses all the above initializations and helper functions to simulate MCM with exploring starts.
        After every num_episodes episodes, we update the policy. We keep doing this until we get the optimal
        policy
    """
    convergence_counter = 0
    iteration = 0
    print(("-"*20+"ITERATION {}"+"-"*20).format(iteration))
    while True:
        # if we have yet to see to consecutive episodes with the same policy
        for episode in range(num_episodes):
            start_state = np.random.choice(start_states)
            print(("-"*20+"EPISODE {}"+"-"*20).format(episode))
            trace, rewards_list = one_episode(start_state, policy, actions)
            rewards = accumulate_rewards(rewards, trace, rewards_list)
        Q = updateQ(Q, rewards)
        print('-'*60)
        print("Q for iteration {}:".format(iteration))
        print(Q[:8])
        previous_policy = policy
        policy = updatePolicy(policy, Q)
        print("POLICY for iteration {}:".format(iteration))
        print(visualize(policy))
        print()
        iteration += 1
        if (policy - previous_policy == np.zeros((9,4))).all():
            # check if the policy changed 
            convergence_counter += 1
        else:
            # once Q changes, reset counter
            convergence_counter = 0 
        if convergence_counter == 1:
            # if policy hasn't changed in the last 10 iterations, we assume it has converged to the optimal policy
            print("OPTIMAL POLICY IS")
            print(visualize(policy))
            break 
        print(("-"*20+"ITERATION {}"+"-"*20).format(iteration))
    

I chose to evaluate and update the policy every 20 episodes

In [19]:
# MCMES1(policy, Q, rewards, 2000)

# Part II

In [20]:
def MCMES2(policy, Q, rewards, num_episodes):
    """ 
        inputs: policy, Q and rewards are the initialized policy, Q and rewards matrix
        This functions uses all the above initializations and helper functions to simulate MCM with exploring starts.
        After every num_episodes episodes, we update the policy. We keep doing this until our Q converges.
        Convergence here is defined as 10 iterations without any change in Q.
    """
    convergence_counter = 0
    iteration = 0
    print(("-"*20+"ITERATION {}"+"-"*20).format(iteration))
    episodes = 0
    while True:
        # if we have yet to see to consecutive episodes with the same policy
        for episode in range(num_episodes):
            start_state = np.random.choice(start_states)
            trace, rewards_list = one_episode(start_state, policy, actions, print_flag=False)
            rewards = accumulate_rewards(rewards, trace, rewards_list)
            episodes += 1
        previous_Q = Q
        Q = updateQ(Q, rewards)
        print("Q for iteration {}:".format(iteration))
        print(Q[:8])
        policy = updatePolicy(policy, Q)
        print("POLICY for iteration {}:".format(iteration))
        print(visualize(policy))
        print()
        iteration += 1
        if (Q - previous_Q == np.zeros((9,4))).all():
            # check if the Q changed 
            convergence_counter += 1
        else:
            # once Q changes, reset counter
            convergence_counter = 0 
        if convergence_counter == 5:
            print("Q converges after {0} iteration(s) and {1} episodes".format(iteration,episodes))
            print("FINAL Q IS")
            print(Q[:8])
            print("FINAL POLICY IS")
            print(visualize(policy))
            break
        print(("-"*20+"ITERATION {}"+"-"*20).format(iteration))
    

Again, I chose to evaluate and update the policy every 100 episodes.

In [21]:
rewards = [[(0,0)for action in range(4)] for state in range(9)]
policy = np.ones((9,4))*0.25
Q = np.ones((9,4))*-9999

In [22]:
MCMES2(policy, Q, rewards,1000)

--------------------ITERATION 0--------------------
Q for iteration 0:
[[-111.53665689  -97.69586375 -100.70301624 -107.53693182]
 [ -99.88980716  -94.08232446  -83.40045767 -113.55927835]
 [ -88.4939759   -89.2808642   -69.16666667  -96.37340153]
 [-105.37864078  -88.22374429  -96.28368794 -101.19788918]
 [ -98.8         -71.39812646  -65.89732143  -96.78894472]
 [ -88.37846154  -59.0798722   -13.45102506  -76.42735043]
 [ -96.38944724  -72.63157895  -97.01904762  -98.11180124]
 [ -77.09243697  -13.5950783   -63.69306931  -88.67192429]]
POLICY for iteration 0:
[['>' 'v' 'v']
 ['>' 'v' 'v']
 ['>' '>' 'T']]

--------------------ITERATION 1--------------------
Q for iteration 1:
[[-111.53665689  -71.91929825 -100.70301624 -107.53693182]
 [ -99.88980716  -94.08232446  -51.51432469 -113.55927835]
 [ -88.4939759   -89.2808642   -51.76897133  -96.37340153]
 [-105.37864078  -60.45496183  -96.28368794 -101.19788918]
 [ -98.8         -71.39812646  -30.88270378  -96.78894472]
 [ -88.37846154  -5